<a href="https://colab.research.google.com/github/joshuaalpuerto/ML-guide/blob/main/RAG_hybrid_Mistral_7B_Instruct.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Loading a GPTQ quantized model requires optimum (`pip install optimum`) and auto-gptq library (`pip install auto-gptq`)
!pip install -q optimum --progress-bar off
!pip install -q auto-gptq --extra-index-url https://huggingface.github.io/autogptq-index/whl/cu118/  --progress-bar off # Use cu117 if on CUDA 11.7
# We need specific transformer to make mistral work
!pip install -q git+https://github.com/huggingface/transformers.git@72958fcd3c98a7afdc61f953aa58c544ebda2f79 --progress-bar off

!pip install -qU langchain Faiss-gpu sentence-transformers
!pip install -q jq # for json loader to work

# !pip install -qU trl Py7zr
!pip install -q rank_bm25
!pip install -q PyPdf

In [ ]:
#connect to google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os

# Unfortunately setting `cache_dir` directly from ctransformers didn't work
# issues : https://github.com/marella/ctransformers/issues/143,
# move cache dir to our repo so later we won't need to load the model again.
# THIS IS NOT WORKING AT THE MOMENT it is a known issue in ctransformers
# We will not use ctransformer as we cant use transformer directly now
os.environ['HF_HOME'] = '/content/drive/MyDrive/huggingface'
os.environ['TRANSFORMERS_CACHE'] = '/content/drive/MyDrive/huggingface/models'

In [ ]:
# @title Load LLM via HF Transformer
from auto_gptq import exllama_set_max_input_length
from langchain.llms.huggingface_pipeline import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

# model_config = {
#     "name": "TheBloke/Mistral-7B-Instruct-v0.1-GPTQ",
#     # 4-bit, with Act Order and group size 32g. Gives highest possible inference quality, with maximum VRAM usage.
#     "revision": "gptq-4bit-32g-actorder_True"
# }

model_config = {
    "name": "TheBloke/Mistral-7B-Instruct-v0.1-GPTQ",
    # 4-bit, with Act Order and group size 32g. Gives highest possible inference quality, with maximum VRAM usage.
    "revision": "gptq-4bit-32g-actorder_True"
}

model = AutoModelForCausalLM.from_pretrained(model_config['name'],
                                             device_map="auto",
                                             trust_remote_code=False,
                                             revision=model_config['revision'])

# Issue to when using 4bit more info here https://github.com/PanQiWei/AutoGPTQ/issues/253
# If using 8bit we don't need this.
# model = exllama_set_max_input_length(model, 4096)

tokenizer = AutoTokenizer.from_pretrained(model_config['name'], use_fast=True)
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    do_sample=True,
    max_new_tokens=4096,
    temperature=0.1,
    top_p=0.95,
    repetition_penalty=1.1
)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
import langchain

from langchain.llms import CTransformers
from langchain.globals import set_llm_cache, set_debug
from langchain.cache import InMemoryCache
from langchain.callbacks import StdOutCallbackHandler
from langchain.chains import LLMChain

llm = HuggingFacePipeline(pipeline=pipe)

def call(prompt, verbose=True, prompt_kwargs=None):
    # Log genereated info
    handler = StdOutCallbackHandler()
    # We use LLMChain + manual retrieval because the answers for our question is inside metadata.
    # There no easyway to do it in Langchain.
    llm_chain = LLMChain(llm=llm, prompt=prompt, verbose=verbose, callbacks=[handler])

    return llm_chain.predict(**prompt_kwargs)



set_llm_cache(InMemoryCache())
set_debug(True)

In [ ]:
from langchain.storage import InMemoryStore
from langchain.embeddings import CacheBackedEmbeddings,HuggingFaceEmbeddings

# In our implementation we have used uses the local file system for storing embeddings and FAISS vector store for retrieval.
# store = LocalFileStore("./cache/")

# We can also set up inmemory cache
# NOTE: we used this as we are more familiar with it
store = InMemoryStore()

embed_model_id="thenlper/gte-large"

# Under the hood HuggingFaceEmbeddings is using sentence-transformer
core_embeddings_model = HuggingFaceEmbeddings(model_name=embed_model_id)

# Here we will leverage a CacheBackedEmbeddings to prevent us from re-embedding similar queries over and over again.
embedder = CacheBackedEmbeddings.from_bytes_store(core_embeddings_model,
                                                  store,
                                                  namespace=embed_model_id)

In [ ]:
from langchain.document_loaders import JSONLoader

# Define the metadata extraction function.
def metadata_func(record: dict, metadata: dict) -> dict:

    metadata["country"] = record.get("country")
    metadata["answer"] = record.get("answer")

    return metadata


loader = JSONLoader(
    file_path='/content/drive/MyDrive/datasets/qna-clean.json',
    jq_schema='.[]',
    content_key="question",
    metadata_func=metadata_func
)

faq_docs = loader.load()

In [ ]:
from langchain.vectorstores import FAISS

# Create VectorStore
vectorstore = FAISS.from_documents(faq_docs, embedder)

In [ ]:
from langchain import PromptTemplate

# Single pass template
# Stuffing all the rules in single prompt confused LLM
# Better to have a route template where depends on user query/question we use specific prompt

PROMPT_TEMPLATE = """<s>[INST] You act as a helpful immigration and relocation agent, responding to questions based on provided context on behalf of the user's assigned agent.

Follow these guidelines for the answer:
- Only answer information related to immigration, relocation, settle-in, cultural integration or similar fields related queries.
- Summarize the answer and make it clear, concise, grammatically correct and direct like a human agent would
- Don't disclose yourself as AI, automated tool or bot, answer like the user's agent would
- Never suggest contacting outside Jobbatical platform. Rewrite the answer to contact the user's agent instead
- write the answer using english language
- Answer the question as truthfully as poosible if you are unsure base on the context below reply with "OUT_OF_CONTEXT"

### Question:
{question}

### Context:
{context}

### Answer:
[/INST]"""

input_variables = ['context', 'question']
custom_prompt = PromptTemplate(template=PROMPT_TEMPLATE,
                            input_variables=input_variables)

# Simple RetrievalQA

`RetrievalQA` from langchain uses a default prompt that takes context and question.

`NOTE:` We cannot use **RetrievalQA** as is. Because the retriever use the `content` of the Document. In our FAQ case our `content` is the `question` and we have the `answer` in the metadata. We did this so we have better semantic similarity (not confusing with the answer). With that said, we need to use custom prompt and `LLMChain` instead.

In [ ]:
from langchain import PromptTemplate


PROMPT_TEMPLATE = """<s>[INST] You are a helpful bot please answer the user question below.

User: {question}
[/INST]"""

custom_prompt = PromptTemplate(template=PROMPT_TEMPLATE,
                            input_variables=['question'])

In [ ]:
question = 'When will I received my TRP?'
custom_prompt.format(question=question)

In [ ]:
from langchain.chains import RetrievalQA
from langchain import PromptTemplate

retriever = vectorstore.as_retriever(search_kwargs={"k":5, 'filter': {'country': 'Estonia'} })
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=retriever,
)
# You can pass prompt directly to RetrievalQA and it will integrate it with it's default prompt
prompt = custom_prompt.format(question=question)
qa_chain(prompt)

# Multi route template

Because if we put all rules in a single prompt LLM get's confused. instead we separated the prompt
- FAQ
- Request for updates

In [ ]:
FAQ_PROMPT_TEMPLATE = """<s>[INST] You act as a helpful immigration and relocation agent, responding to questions based on provided context on behalf of the user's assigned agent.

Follow these guidelines for the answer:
- Only answer information related to immigration, relocation, settle-in, cultural integration or similar fields related queries.
- Summarize the answer and make it clear, concise, grammatically correct and direct like a human agent would
- Don't disclose yourself as AI, automated tool or bot, answer like the user's agent would
- Never suggest contacting outside Jobbatical platform. Rewrite the answer to contact the user's agent instead
- write the answer using english language
- You should only base your answer base on the context below. If you can't answer the question then reply with "OUT_OF_CONTEXT"

### Question:
{question}

### Context:
{context}

### Answer:
[/INST]"""

REQUESTING_UPDATES_PROMPT_TEMPLATE = """<s>[INST] You act as a helpful immigration and relocation agent, responding to questions on behalf of the user's assigned agent.

Follow these guidelines for the answer:
- For questions that are specific about their own or family member's immigration case progress, suggest to contact their agents.
- Don't use any general information.
- write the answer using english language


### Question:
{question}

### Answer:
[/INST]"""


In [ ]:
prompt_infos = [
    {
        "name": "faq",
        "description": "Good for answering generic immigration or relocation questions",
        "prompt_template": FAQ_PROMPT_TEMPLATE,
    },
    {
        "name": "request for updates",
        "description": "Good for answering questions related to user's relocation process or updates",
        "prompt_template": REQUESTING_UPDATES_PROMPT_TEMPLATE,
    },
]

prompts_dict = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompts_dict[name] = PromptTemplate(template=prompt_template, input_variables = ['context', 'question'])

In [ ]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser

# We modify the prompt because we are using Mistral
# original prompt https://github.com/langchain-ai/langchain/blob/8c150ad7f6e9073f3508187ce79ea4715b74e105/libs/langchain/langchain/chains/router/multi_prompt_prompt.py#L4
MULTI_PROMPT_ROUTER_TEMPLATE = """<s>[INST]
Given a raw text input to a language model select the model prompt best suited for \
the input. You will be given the names of the available prompts and a description of \
what the prompt is best suited for. You may also revise the original input if you \
think that revising it will ultimately lead to a better response from the language \
model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \\ name of the prompt to use or "DEFAULT"
    "next_inputs": string \\ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt names specified below OR \
it can be "DEFAULT" if the input is not well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input if you don't think any \
modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}


<< INPUT >>
{{input}}

<< OUTPUT (must include ```json at the start of the response) >>
<< OUTPUT (must end with ```) >>[/INST]
"""


destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(destinations=destinations_str)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["question"],
    output_parser=RouterOutputParser(),
)
router_chain = LLMRouterChain.from_llm(llm, router_prompt)

router_chain('Any update for my visa?')

/usr/local/lib/python3.10/dist-packages/langchain/chains/llm.py:280: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


{'input': 'Any update for my visa?',
 'destination': 'request for updates',
 'next_inputs': {'input': 'Any update for my visa?'}}

# Using LLM Extractor to filter documents base on relevance.

NOTE: we didn't use LLMChainExtractor because our embeddings only consist of question. The context is in metadata.

there's no easy way of doing it with Langchain using `LLMChainExtractor`

NOTE: this is not working properly :(
  I tried with OPENAI this prompt is working fine.

In [ ]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor
from langchain import PromptTemplate

# Naive top_k retriever (there a chance it gets incorrect context)
def get_documents_as_context(query, retriever):
    docs = retriever.get_relevant_documents(query)
    inputs = [f"- {doc.metadata['answer']}" for doc in docs]
    context = "\n".join(inputs)
    return context

PROMPT_EXTRACTOR = """<s>[INST] Given the following question and context, extract any part of the context *AS IS* that is relevant to answer the question. If none of the context is relevant return {no_output_str}.

Remember, *DO NOT* edit the extracted parts of the context.

> Question: {question}
> Context:
>>>
{context}
>>>
Extracted relevant parts: [/INST]"""

def llm_chain_extractor(llm, query, retriever):
    context = get_documents_as_context(query, retriever)
    compressor = LLMChainExtractor.from_llm(llm, prompt=PromptTemplate(template=PROMPT_EXTRACTOR,
                            input_variables=['question', 'context']))
    return compressor.llm_chain.predict_and_parse(question=query, context=context, )

query = "do you have any update for my visa?"
retriever = vectorstore.as_retriever(search_kwargs={"k":5, 'filter': {'country': 'Estonia'} })
llm_chain_extractor(llm, query, retriever)


/usr/local/lib/python3.10/dist-packages/langchain/chains/llm.py:280: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


[chain/start] [1:chain:LLMChain] Entering Chain run with input:
{
  "question": "do you have any update for my visa?",
  "context": "- When applying at an Estonian foreign representation or at the Police Board in Estonia, Jobbatical will pay the state fee on your behalf and the payment order will be uploaded on the platform.\nIf you're applying independently and/or via VFS Global, you will need to pay the state fee yourself at the VFS centre. In this case, please check with the relevant centre to see the requirements for paying the state fee.\n- No, D visa cannot be extended, rather you may be eligible to apply for a second consecutive D visa. The duration of a new D visa depends on the duration of the previous D visa - in case of two consecutive visas, the total duration cannot exceed 548 days within a 730 day period."
}
[llm/start] [1:chain:LLMChain > 2:llm:CTransformers] Entering LLM run with input:
{
  "prompts": [
    "<s>[INST] Given the following question and context, extract an

' <<no_output_str>>'

In [ ]:
import langchain
from langchain.chains import LLMChain
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import EmbeddingsFilter
from langchain.callbacks import StdOutCallbackHandler

# A bit slower but this improves retrieval by using bruteforce similarity(cos_sim) against user query.
def filter_similar_docs_from_query(query, embeddings, retriever, similarity_threshold=0.90):
  embeddings_filter = EmbeddingsFilter(embeddings=embeddings, similarity_threshold=similarity_threshold)
  compression_retriever = ContextualCompressionRetriever(base_compressor=embeddings_filter, base_retriever=retriever)

  docs = compression_retriever.get_relevant_documents(query)
  inputs = [f"- {doc.metadata['answer']}" for doc in docs]
  context = "\n".join(inputs)

  return context

def llm_call(question, context, prompt):
  # Log genereated info
  handler = StdOutCallbackHandler()
  # We use LLMChain + manual retrieval because the answers for our question is inside metadata.
  # There no easyway to do it in Langchain.
  llm_chain = LLMChain(
      llm=llm,
      prompt=prompt,
      verbose=True,
      callbacks=[handler]
  )

  return llm_chain.predict(context=context, question=query)

def get_prompt_by_query(query, default_prompt = prompts_dict['faq']):
  prompt_router = router_chain(query)
  return prompts_dict[prompt_router['destination']] or default_prompt


def inference(query, country):
  retriever = vectorstore.as_retriever(search_kwargs={"k":5, 'filter': {'country': country} })
  context = filter_similar_docs_from_query(query=query, embeddings=core_embeddings_model, retriever=retriever)
  prompt = get_prompt_by_query(query)

  return llm_call(question=query, context=context, prompt=prompt)
# To print the final prompt
  langchain.debug = True

In [ ]:
%%time

# query = "when will I received my TRP?"
# query = "Me and my wife doesn't marriage certificate yet. Is that a problem?"
query = "Who is michel jordan?"
country = "Estonia"

inference(query=query, country=country)

/usr/local/lib/python3.10/dist-packages/langchain/chains/llm.py:280: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(




> Entering new LLMChain chain...
Prompt after formatting:
<s>[INST] You act as a helpful immigration and relocation agent, responding to questions based on provided context on behalf of the user's assigned agent.

Follow these guidelines for the answer:
- Only answer information related to immigration, relocation, settle-in, cultural integration or similar fields related queries.
- Summarize the answer and make it clear, concise, grammatically correct and direct like a human agent would
- Don't disclose yourself as AI, automated tool or bot, answer like the user's agent would
- Never suggest contacting outside Jobbatical platform. Rewrite the answer to contact the user's agent instead
- write the answer using english language
- You should only base your answer base on the context below. If you can't answer the question then reply with "OUT_OF_CONTEXT" 

### Question: 
Who is michel jordan?

### Context:


### Answer: 
[/INST]

> Finished chain.
CPU times: user 3.85 s, sys: 21.7 ms, t

' OUT\\_OF\\_CONTEXT'

In [ ]:
%%time

query = "When will I received my TRP?"
query = "Good internet provider?"
country = "Germany"

inference(query=query, country=country)

/usr/local/lib/python3.10/dist-packages/langchain/chains/llm.py:280: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(




> Entering new LLMChain chain...
Prompt after formatting:
<s>[INST] You act as a helpful immigration and relocation agent, responding to questions based on provided context on behalf of the user's assigned agent.

Follow these guidelines for the answer:
- Only answer information related to immigration, relocation, settle-in, cultural integration or similar fields related queries.
- Summarize the answer and make it clear, concise, grammatically correct and direct like a human agent would
- Don't disclose yourself as AI, automated tool or bot, answer like the user's agent would
- Never suggest contacting outside Jobbatical platform. Rewrite the answer to contact the user's agent instead
- write the answer using english language
- You should only base your answer base on the context below. If you can't answer the question then reply with "OUT_OF_CONTEXT" 

### Question: 
Good internet provider?

### Context:


### Answer: 
[/INST]

> Finished chain.
CPU times: user 1.84 s, sys: 16.5 ms,

' OUT\\_OF\\_CONTEXT'

## Debugging

> NOTE: You don't need to run this

In [ ]:
retriever = vectorstore.as_retriever(search_kwargs={"k":5, 'filter': {'country': 'Germany'} })
filter_similar_docs_from_query(query="Am I eligible for Blue card?",
                               embeddings=core_embeddings_model,
                               retriever=retriever, similarity_threshold=0.89)

"- No occupational groups are excluded, as long as the necessary conditions are met. The employment does have to correspond to the qualification.\n- If you are already in Germany with a German residence title, you can apply for an EU Blue Card at the foreigner's registration office. If you're currently abroad, you will need to apply at your nearest German diplomatic mission for a visa for the purpose of gainful employment before entering Germany. After entering Germany, you can apply for an EU Blue Card at the foreigners' registration office. Citizens of Australia, Israel, Japan, Canada, the Republic of Korea, New Zealand, and the United States of America can enter Germany without a visa and apply for the EU Blue Card within three months of entering. People who enter the country without a visa are generally not allowed to work directly. This is only allowed when they have received a residence title that allows gainful employment. Since it can take several weeks to get one, we recommend